# Deep Learning Tutorial - Modeling

In [1]:
import numpy as np
import pprint

In [18]:
INPUT_SIZE = 2
LAYER_SIZES = [5, 5, 1]
LAYER_SIZES.insert(0, INPUT_SIZE)
LAYER_ACTIVATIONS = ['relu', 'relu', 'sigmoid']

In [38]:
def initialize_network():
    architecture = {}
    for layer in range(1, len(LAYER_SIZES)):
        architecture[f'layer_{layer}'] = {
            'w': np.random.randn(LAYER_SIZES[layer],
                                 LAYER_SIZES[layer-1]) * 1,
            'b': np.zeros(layer),
            'activation': LAYER_ACTIVATIONS[layer-1]
        }
    return architecture

In [39]:
network = initialize_network()
pprint.pprint(network)

{'layer_1': {'activation': 'relu',
             'b': array([0.]),
             'w': array([[ 0.16265908, -0.12909023],
       [ 0.81427627,  1.31800588],
       [-0.01846787, -0.94813517],
       [-1.12477662, -0.01261812],
       [ 0.21627657,  0.94370082]])},
 'layer_2': {'activation': 'relu',
             'b': array([0., 0.]),
             'w': array([[-1.44603078, -1.00208109,  0.66925218, -0.07593184,  1.23347454],
       [-0.44357222, -1.15782194, -0.90132038, -0.30412108, -0.07512328],
       [-0.62317037, -1.6291713 , -1.46889242,  2.080184  , -0.2964723 ],
       [-1.42418382, -1.33797094, -0.55368748, -1.7519944 , -0.11157937],
       [ 2.68161409,  0.97724664, -0.60448605,  0.11955348, -1.56750555]])},
 'layer_3': {'activation': 'sigmoid',
             'b': array([0., 0., 0.]),
             'w': array([[ 0.34481762,  0.31121743, -1.09858042, -1.39811787, -0.55488354]])}}


In [5]:
def sigmoid_activation(Z):
    activation = 1/(1 + np.exp(-1*Z))
    return activation

In [ ]:
def softmax_activation(Z):
    activation = np.exp(Z) / np.sum(np.exp(Z))
    return activation

In [6]:
def relu_activation(Z):
    activation = np.max(0, Z)
    return activation

In [7]:
def dZ_sigmoid(dA, Z):
    sigmoid = sigmoid_activation(Z)
    dZ = dA * sigmoid * (1 - sigmoid)
    return dZ

In [41]:
def dZ_softmax(dA, Z):
    softmax = softmax_activation(Z)
    softmax_matrix = np.tile(softmax)
    dZ = np.diag(softmax) - (softmax_matrix*np.transpose(softmax_matrix))

In [8]:
def dZ_relu(dA, Z):
    dZ = np.copy(dA)
    dZ[Z <= 0] = 0
    return dZ

In [9]:
act_map = {
    'sigmoid': sigmoid_activation,
    'relu': relu_activation,
    'softmax': softmax_activation
}

In [10]:
dZ_map = {
    'sigmoid': dZ_sigmoid,
    'relu': dZ_relu,
    'softmax': dZ_softmax
}

In [11]:
def single_forward_pass(A_previous, W, b, activation):
    try:
        act_function = act_map[activation]
    except KeyError:
        print(f'The activation {activation} is not recognized.\nIt must be one of the following: {list(act_map.keys())}')
        return None
    
    Z = np.dot(W, A_previous) + b
    A = act_function(Z)
    
    return A, Z

In [40]:
def full_forward_pass(X, network):
    
    cache = {}
    A = X
    
    for layer in range(1, len(network) + 1):
        
        A_previous = A
        A, Z = single_forward_pass(A_previous, network[layer]['W'], network[layer]['b'], network[layer]['activation'])
        
        cache[f'A_{layer-1}'] = A_previous
        cache[f'Z_{layer}'] = Z
        
    return A, cache

In [12]:
def compute_cross_entropy_cost(y_pred, y):
    
    cost = np.sum(-1*(y * np.log(y_pred)))
    
    return cost

In [13]:
def single_backward_pass():
    return None

In [14]:
def full_backward_pass():
    return None

In [15]:
def train_nn():
    return None

### Resources
This notebook has been inspired by the Towards Data Science post [Let’s code a Neural Network in plain NumPy](https://towardsdatascience.com/lets-code-a-neural-network-in-plain-numpy-ae7e74410795).

* [The Softmax Function Derivative (Part 1)](https://aimatters.wordpress.com/2019/06/17/the-softmax-function-derivative/).